In [ ]:
import xarray as xr

import acspype.processing as acsproc
import acspype.qaqc as acsqaqc
from acspype import ACSTSCor
from acspype.utils.ooi import get_ooi_optaa_cal, reformat_ooi_optaa

## Acquire a File From CE02SHSM Deployment 9

1. First we import the dataset from OpenDAP using Xarray.
2. Then we reformat the dataset to be compatible with acspype using `reformat_ooi_optaa`.
    -   No ACS data are removed from the dataset, only excess variables (e.g. pressure_counts) and timestamps (e.g. port_timestamp). These are either not needed for processing or are meaningless.
    - Some variables are compressed and assigned reassigned dimensions (e.g. lat, lon, deployment) since they should be the same throughout a single file. 
    - File and variable attributes are preserved.
    - Attributes and variables are alphabetized.
3. `reformat_ooi_optaa` renames `optical_absorption` and `beam_attenuation` to `a_m_discontinuity` and `c_m_discontinuity`, respectively, since OOI does not correct for the discontinuity jump inherent in all ACS'. We can drop these because we will reprocess to that stage as part of this example anyway.
4. Since `reformat_ooi_optaa` adds the dimensions of latitude, longitude, and deployment, and we are only working with a single deployment file, we can subset by those values.

In [ ]:
opendap_url = "http://thredds.dataexplorer.oceanobservatories.org/thredds/dodsC/ooigoldcopy/public/CE02SHSM-RID27-01-OPTAAD000-recovered_host-optaa_dj_dcl_instrument_recovered/deployment0009_CE02SHSM-RID27-01-OPTAAD000-recovered_host-optaa_dj_dcl_instrument_recovered_20190622T003023-20190723T233200.492000.nc"
ds = xr.open_dataset(opendap_url)

ds = reformat_ooi_optaa(ds)

ds = ds.drop_vars(['a_m_discontinuity', 'c_m_discontinuity'], errors='ignore')  

ds = ds.sel(latitude = float(ds.latitude), longitude = float(ds.longitude), deployment = int(ds.deployment))

## Acquire the Device Factory Calibration Information and TS Correction Coefficients
The factory calibration information is available through the OOI REST API. The `get_ooi_optaa_cal` function retrieves this information and reformats it to match the format of acspype's `ACSDev` class. The `ACSTSCor` class imports coefficients that have been pre-compiled to a Python file within acspype from the original TS4.cor file that accompanies each device. These files should be the same between all ACS.
If using acspype to process ACS data, you do not need to import the TS4.cor file on your own, but that functionality exists through the `ACSTS4CorReader` class if you want to do that yourself.

In [ ]:
dev = get_ooi_optaa_cal(ds)

In [ ]:
tscor = ACSTSCor()

## Reprocess the Data

In [ ]:
ds['internal_temperature'] = acsproc.compute_internal_temperature(ds['raw_internal_temperature'])
ds['external_temperature'] = acsproc.compute_external_temperature(ds['raw_external_temperature'])

In [ ]:
ds['a_uncorrected'] = acsproc.compute_uncorrected(ds['a_signal'], ds['a_reference'], dev)
ds['c_uncorrected'] = acsproc.compute_uncorrected(ds['c_signal'], ds['c_reference'], dev)

In [ ]:
ds['a_m_discontinuity'] = acsproc.compute_measured(ds['a_uncorrected'], 'a', ds['internal_temperature'], dev)
ds['c_m_discontinuity'] = acsproc.compute_measured(ds['c_uncorrected'], 'c', ds['internal_temperature'], dev)

In [ ]:
discontinuity_index = acsproc.find_discontinuity_index(ds['a_wavelength'], ds['c_wavelength'])
ds['a_m'], ds['a_discontinuity_offset'] = acsproc.discontinuity_correction(ds.a_m_discontinuity, 'a_wavelength', discontinuity_index)
ds['c_m'], ds['c_discontinuity_offset'] = acsproc.discontinuity_correction(ds.c_m_discontinuity, 'c_wavelength', discontinuity_index)

In [ ]:
ds['a_mts'] = acsproc.ts_correction(ds.a_m, 'a', ds.sea_water_temperature, ds.sea_water_practical_salinity, dev, tscor)
ds['c_mts'] = acsproc.ts_correction(ds.c_m, 'c', ds.sea_water_temperature, ds.sea_water_practical_salinity, dev, tscor)

In [ ]:
ds['a_mts'] = acsproc.zero_shift(ds.a_mts)
ds['c_mts'] = acsproc.zero_shift(ds.c_mts)

## Flag Data

In [ ]:
ds['flag_internal_temperature'] = acsqaqc.internal_temperature_test(ds['internal_temperature'], dev)